### Tentando implementar a estabilidade

In [3]:
import random
import numpy as np
import pandas as pd

# Função para calcular o centro de massa
def calcular_centro_massa(containers):
    total_peso = 0
    soma_peso_x = 0
    soma_peso_y = 0
    soma_peso_z = 0
    
    for peso, x, y, z in containers:
        soma_peso_x += peso * x
        soma_peso_y += peso * y
        soma_peso_z += peso * z
        total_peso += peso
    
    centro_massa_x = soma_peso_x / total_peso
    centro_massa_y = soma_peso_y / total_peso
    centro_massa_z = soma_peso_z / total_peso
    
    return (centro_massa_x, centro_massa_y, centro_massa_z)

# Função para avaliar a estabilidade
def avaliar_estabilidade(centro_massa):
    centro_ideal = (1.5, 1.5, 1.5)  # Centro geométrico ideal
    distancia = sum((cm - ci)**2 for cm, ci in zip(centro_massa, centro_ideal)) ** 0.5
    return distancia

# Função modificada para incluir cálculo de estabilidade
def preencher_porto(df_cromossomo):
    porto = np.zeros((2, 5, 3), dtype=int)
    balsa = np.zeros((3, 4, 3), dtype=int)
    porto_preenchido = np.ones((2, 5, 3), dtype=int)
    cal_movimentos = 0
    
    ordem_inicial = []
    containers = []

    df_cromossomo = df_cromossomo.sort_values(by='Posição Inicial')
    for _, linha in df_cromossomo.iterrows():
        ordem_inicial.append(linha['Container'])
    idx = 0
    for andar in range(2):
        for x in range(5):
            for y in range(3):
                if idx < 30:
                    porto[andar, x, y] = ordem_inicial[idx]
                    idx += 1
    
    df_cromossomo = df_cromossomo.sort_values(by='Ordem de Movimentação')
    
    for _, linha in df_cromossomo.iterrows():
        container = linha['Container']
        pos_inicial = linha['Posição Inicial'] 
        pos_final = linha['Posição Final'] 
        
        andar_ini, x_ini, y_ini = pos_inicial // 15, (pos_inicial % 15) // 3, (pos_inicial % 15) % 3
        andar_fin, x_fin, y_fin = pos_final // 12, (pos_final % 12) // 3, (pos_final % 12) % 3
        
        errou = 0
        pode_mover = True

        if errou == 0:
            if andar_ini < 1 and porto_preenchido[andar_ini + 1, x_ini, y_ini] == 1:
                errou = 1
                pode_mover = False
            
            if andar_fin > 0 and balsa[andar_fin - 1, x_fin, y_fin] == 0:
                errou = 1
                pode_mover = False

            if balsa[andar_fin, x_fin, y_fin] == 1:
                errou = 1
                pode_mover = False
                    
            if pode_mover:
                porto_preenchido[andar_ini, x_ini, y_ini] = 0
                balsa[andar_fin, x_fin, y_fin] = 1
                
                cal_movimentos += 2
                cal_movimentos += 0 if x_ini == x_fin else 1
                cal_movimentos += 0 if y_ini == y_fin else 1
                
                peso_container = 10
                containers.append((peso_container, x_fin, y_fin, andar_fin + 1))

                if andar_fin > 0 and np.any(balsa[andar_fin - 1, :, :] == 0):
                    cal_movimentos += 200

                centro_massa = calcular_centro_massa(containers)
                estabilidade = avaliar_estabilidade(centro_massa)
                if estabilidade > 2:
                    errou = 1
                    pode_mover = False
                else:  
                    cal_movimentos+= estabilidade*100    
            else:
                cal_movimentos += 1000
        else:
            cal_movimentos += 1000

    return cal_movimentos

# Funções do algoritmo genético (permanecem iguais, apenas chamam a função modificada)
def gerar_cromossomo():
    cromossomo = []
    
    posicoes_iniciais = list(range(30))
    random.shuffle(posicoes_iniciais)
    cromossomo.extend(posicoes_iniciais)

    posicoes_finais = list(range(36))
    random.shuffle(posicoes_finais)
    posicoes_finais = posicoes_finais[:30]
    cromossomo.extend(posicoes_finais)
    
    ordem_movimentacao = list(range(30))
    random.shuffle(ordem_movimentacao)
    cromossomo.extend(ordem_movimentacao)
    
    return cromossomo

def cromossomo_para_dataframe(cromossomo):
    container = list(range(30))
    posicao_inicial = cromossomo[:30]
    posicao_final = cromossomo[30:60]
    ordem_movimentacao = cromossomo[60:90]
    
    df = pd.DataFrame({
        'Container': container,
        'Posição Inicial': posicao_inicial,
        'Posição Final': posicao_final,
        'Ordem de Movimentação': ordem_movimentacao
    })
        
    df = df.sort_values(by='Ordem de Movimentação')
    
    return df

def inicializar_populacao(tamanho_populacao):
    populacao = []
    for _ in range(tamanho_populacao):
        cromossomo = gerar_cromossomo()
        populacao.append(cromossomo)
    return populacao

def verifica_repetidos(cromossomo):
    intervalos = [
        (0, 29),
        (30, 59),
        (60, 89)
    ]
    
    for intervalo in intervalos:
        valores = cromossomo[intervalo[0]:intervalo[1]+1]
        if len(valores) != len(set(valores)):
            return True
    
    return False

def selecao_roleta(populacao, fitness):
    soma_fitness = sum(fitness)
    pick = random.uniform(0, soma_fitness)
    atual = 0
    for cromossomo, fit in zip(populacao, fitness):
        atual += fit
        if atual >= pick:
            return cromossomo

def cruzamento_pmx(pai1, pai2, inicio, fim):
    tamanho = fim - inicio + 1
    filho1 = pai1[:inicio] + [-1] * tamanho + pai1[fim+1:]
    filho2 = pai2[:inicio] + [-1] * tamanho + pai2[fim+1:]
    
    mapeamento1 = pai1[inicio:fim+1]
    mapeamento2 = pai2[inicio:fim+1]
    
    filho1[inicio:fim+1] = mapeamento2
    filho2[inicio:fim+1] = mapeamento1
    
    for i in range(inicio):
        if filho1[i] in mapeamento2:
            idx = mapeamento2.index(filho1[i])
            while mapeamento1[idx] in mapeamento2:
                idx = mapeamento2.index(mapeamento1[idx])
            filho1[i] = mapeamento1[idx]
    
    for i in range(fim+1, len(pai1)):
        if filho1[i] in mapeamento2:
            idx = mapeamento2.index(filho1[i])
            while mapeamento1[idx] in mapeamento2:
                idx = mapeamento2.index(mapeamento1[idx])
            filho1[i] = mapeamento1[idx]
    
    for i in range(inicio):
        if filho2[i] in mapeamento1:
            idx = mapeamento1.index(filho2[i])
            while mapeamento2[idx] in mapeamento1:
                idx = mapeamento1.index(mapeamento2[idx])
            filho2[i] = mapeamento2[idx]
    
    for i in range(fim+1, len(pai2)):
        if filho2[i] in mapeamento1:
            idx = mapeamento1.index(filho2[i])
            while mapeamento2[idx] in mapeamento1:
                idx = mapeamento1.index(mapeamento2[idx])
            filho2[i] = mapeamento2[idx]
    
    return filho1, filho2

def cruzamento(pai, mae):
    filho1, filho2 = pai[:], mae[:]
    
    ponto_corte1 = random.randint(0, 29)
    ponto_corte2 = random.randint(0, 29)
    inicio, fim = min(ponto_corte1, ponto_corte2), max(ponto_corte1, ponto_corte2)
    filho1[:30], filho2[:30] = cruzamento_pmx(pai[:30], mae[:30], inicio, fim)
    
    ponto_corte1 = random.randint(30, 59)
    ponto_corte2 = random.randint(30, 59)
    inicio, fim = min(ponto_corte1, ponto_corte2) - 30, max(ponto_corte1, ponto_corte2) - 30
    filho1[30:60], filho2[30:60] = cruzamento_pmx(pai[30:60], mae[30:60], inicio, fim)
    
    ponto_corte1 = random.randint(60, 89)
    ponto_corte2 = random.randint(60, 89)
    inicio, fim = min(ponto_corte1, ponto_corte2) - 60, max(ponto_corte1, ponto_corte2) - 60
    filho1[60:90], filho2[60:90] = cruzamento_pmx(pai[60:90], mae[60:90], inicio, fim)
    
    return filho1, filho2

def mutacao(cromossomo, taxa_mutacao=0.01):
    if random.random() < taxa_mutacao:
        part = random.randint(0, 2)
        idx1, idx2 = random.sample(range(30), 2)
        if part == 0:
            cromossomo[idx1], cromossomo[idx2] = cromossomo[idx2], cromossomo[idx1]
        elif part == 1:
            cromossomo[30+idx1], cromossomo[30+idx2] = cromossomo[30+idx2], cromossomo[30+idx1]
        else:
            cromossomo[60+idx1], cromossomo[60+idx2] = cromossomo[60+idx2], cromossomo[60+idx1]
    return cromossomo

def algoritmo_genetico(tamanho_populacao, taxa_mutacao):
    populacao = inicializar_populacao(tamanho_populacao)
    fitness = [preencher_porto(cromossomo_para_dataframe(cromossomo)) for cromossomo in populacao]
    
    geracao = 0
    sem_melhoria = 0
    melhor_fit = min(fitness)
    
    while sem_melhoria < 40:
        geracao += 1
        nova_populacao = []
        while len(nova_populacao) < tamanho_populacao:
            pai = selecao_roleta(populacao, fitness)
            mae = selecao_roleta(populacao, fitness)
            
            filho1, filho2 = cruzamento(pai, mae)
            filho1 = mutacao(filho1, taxa_mutacao)
            filho2 = mutacao(filho2, taxa_mutacao)
            
            nova_populacao.append(filho1)
            nova_populacao.append(filho2)
        
        populacao.extend(nova_populacao)
        fitness = [preencher_porto(cromossomo_para_dataframe(cromossomo)) for cromossomo in populacao]
        
        populacao_ordenada = sorted(zip(populacao, fitness), key=lambda x: x[1])
        
        melhor_5_porcento = int(tamanho_populacao * 0.05)
        nova_metade = tamanho_populacao - melhor_5_porcento
        
        populacao = [cromossomo for cromossomo, _ in populacao_ordenada[:melhor_5_porcento]] + [cromossomo for cromossomo, _ in populacao_ordenada[melhor_5_porcento:melhor_5_porcento+nova_metade]]
        fitness = [fit for _, fit in populacao_ordenada[:melhor_5_porcento]] + [fit for _, fit in populacao_ordenada[melhor_5_porcento:melhor_5_porcento+nova_metade]]
        
        nova_melhor_fit = min(fitness)
        print(f"Geração {geracao}: Melhor fitness = {nova_melhor_fit}")

        if nova_melhor_fit < melhor_fit:
            melhor_fit = nova_melhor_fit
            sem_melhoria = 0
        else:
            sem_melhoria += 1
    
    melhor_cromossomo = populacao[fitness.index(min(fitness))]
    return melhor_cromossomo, min(fitness)

# Configurações
tamanho_populacao = 100
taxa_mutacao = 0.1

melhor_cromossomo, melhor_fit = algoritmo_genetico(tamanho_populacao, taxa_mutacao)
print(f"Melhor cromossomo encontrado: {melhor_cromossomo}")
print(f"Melhor fitness: {melhor_fit}")


Geração 1: Melhor fitness = 16594.470359578037
Geração 2: Melhor fitness = 16594.470359578037
Geração 3: Melhor fitness = 15875.774214853971
Geração 4: Melhor fitness = 15875.774214853971
Geração 5: Melhor fitness = 15220.913260274247
Geração 6: Melhor fitness = 15220.913260274247
Geração 7: Melhor fitness = 15220.913260274247
Geração 8: Melhor fitness = 15220.913260274247
Geração 9: Melhor fitness = 15220.913260274247
Geração 10: Melhor fitness = 14481.421182145215
Geração 11: Melhor fitness = 14481.421182145215
Geração 12: Melhor fitness = 14481.421182145215
Geração 13: Melhor fitness = 14481.421182145215
Geração 14: Melhor fitness = 14481.421182145215
Geração 15: Melhor fitness = 14481.421182145215
Geração 16: Melhor fitness = 14481.421182145215
Geração 17: Melhor fitness = 14481.421182145215
Geração 18: Melhor fitness = 12219.470263002017
Geração 19: Melhor fitness = 12219.470263002017
Geração 20: Melhor fitness = 12219.470263002017
Geração 21: Melhor fitness = 12219.470263002017
G

#### Verifica se ha numeros repetidos


In [6]:
def verifica_repetidos(cromossomo):
    intervalos = [
        (0, 29),
        (30, 59),
        (60, 89)
    ]
    
    for intervalo in intervalos:
        valores = cromossomo[intervalo[0]:intervalo[1]+1]
        if len(valores) != len(set(valores)):
            return True
    
    return False

if verifica_repetidos(melhor_cromossomo):
    print("Valores repetidos encontrados!")
else:
    print("Não há valores repetidos.")

df_cromossomo = cromossomo_para_dataframe(melhor_cromossomo)
print("DataFrame do Cromossomo:")
print(df_cromossomo)


Não há valores repetidos.
DataFrame do Cromossomo:
    Container  Posição Inicial  Posição Final  Ordem de Movimentação
26         26               23              8                      0
3           3                7             24                      1
14         14               19              4                      2
23         23                4             16                      3
9           9               20              5                      4
0           0               29             11                      5
17         17               21              7                      6
7           7               14              2                      7
4           4               25             10                      8
29         29               16              1                      9
24         24               18              6                     10
20         20                3              3                     11
18         18               24              9       

### Simular Melhor cromomossomo

In [7]:
def Simular_preencher_porto(df_cromossomo):
    # Inicializar o porto e balsa
    porto = np.zeros((2, 5, 3), dtype=int)
    balsa = np.zeros((3, 4, 3), dtype=int)
    porto_preenchido = np.ones((2, 5, 3), dtype=int)
    cal_movimentos = 0
    
    # Preencher o porto com os dados do cromossomo
    ordem_inicial = []
    ordem_final = []

    df_cromossomo = df_cromossomo.sort_values(by='Posição Inicial')
    for _, linha in df_cromossomo.iterrows():
        ordem_inicial.append(linha['Container'])
    idx = 0
    for andar in range(2):
        for x in range(5):
            for y in range(3):
                if idx < 30:
                    porto[andar, x, y] = ordem_inicial[idx]
                    idx += 1
    
    df_cromossomo = df_cromossomo.sort_values(by='Ordem de Movimentação')
    containers = []
    
    # Realizar movimentações
    for _, linha in df_cromossomo.iterrows():
        container = linha['Container']
        pos_inicial = linha['Posição Inicial'] 
        pos_final = linha['Posição Final'] 
        
        andar_ini, x_ini, y_ini = pos_inicial // 15, (pos_inicial % 15) // 3, (pos_inicial % 15) % 3
        andar_fin, x_fin, y_fin = pos_final // 12, (pos_final % 12) // 3, (pos_final % 12) % 3
        
        errou = 0
        pode_mover = True

        if (errou==0):
            
            # Verificar se podemos remover do porto preenchido
            if andar_ini < 1:
                if porto_preenchido[andar_ini + 1, x_ini, y_ini] == 1:
                    errou =1
                    pode_mover = False
            
            # Verificar se podemos adicionar à balsa
            if andar_fin > 0:
                if balsa[andar_fin - 1, x_fin, y_fin] == 0:
                    errou =1
                    pode_mover = False

            # Verifica se o lugar está ocupado
            if balsa[andar_fin , x_fin, y_fin] == 1:
                errou =1
                pode_mover = False

                    
            if pode_mover:
                # Atualizar porto_preenchido e balsa
                porto_preenchido[andar_ini, x_ini, y_ini] = 0
                balsa[andar_fin, x_fin, y_fin] = 1
                
                cal_movimentos += 2 #alturas subir e descer
                cal_movimentos += 0 if x_ini == x_fin else 1  # verificação x
                cal_movimentos += 0 if y_ini == y_fin else 1  # verificação y
                
                # Adicionar contêiner à lista
                peso_container = 10  # Supondo peso fixo de 10 toneladas por contêiner
                containers.append((peso_container, x_fin, y_fin, 1))  # Altura simplificada

                # Calcular centro de massa e avaliar estabilidade
                centro_massa = calcular_centro_massa(containers)
                estabilidade = avaliar_estabilidade(centro_massa)

                print(f"Movendo container {container} de ({pos_inicial})({andar_ini}, {x_ini}, {y_ini}) para ({pos_final})({andar_fin}, {x_fin}, {y_fin})")
                print(f"Estabilidade {estabilidade}: X->({centro_massa[0]}) Y->({centro_massa[1]}) Z->({centro_massa[2]})")
                print("Estado atual do Porto Preenchido:")
                print(porto_preenchido)
                print("Estado atual da Balsa:")
                print(balsa)
                print("\n")

                if estabilidade > 2:  # Ajuste o valor do limite conforme necessário
                    errou =1
                    pode_mover = False # Penalidade para cromossomos instáveis
            else:
                print(f"não moveu container {container} de ({pos_inicial})({andar_ini}, {x_ini}, {y_ini}) para ({pos_final})({andar_fin}, {x_fin}, {y_fin})")
                cal_movimentos += 1000
        else:
            cal_movimentos += 1000


Simular_preencher_porto(cromossomo_para_dataframe(melhor_cromossomo))

Movendo container 26 de (23)(1, 2, 2) para (8)(0, 2, 2)
Estabilidade 0.8660254037844386: X->(2.0) Y->(2.0) Z->(1.0)
Estado atual do Porto Preenchido:
[[[1 1 1]
  [1 1 1]
  [1 1 1]
  [1 1 1]
  [1 1 1]]

 [[1 1 1]
  [1 1 1]
  [1 1 0]
  [1 1 1]
  [1 1 1]]]
Estado atual da Balsa:
[[[0 0 0]
  [0 0 0]
  [0 0 1]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]]]


não moveu container 3 de (7)(0, 2, 1) para (24)(2, 0, 0)
Movendo container 14 de (19)(1, 1, 1) para (4)(0, 1, 1)
Estabilidade 0.5: X->(1.5) Y->(1.5) Z->(1.0)
Estado atual do Porto Preenchido:
[[[1 1 1]
  [1 1 1]
  [1 1 1]
  [1 1 1]
  [1 1 1]]

 [[1 1 1]
  [1 0 1]
  [1 1 0]
  [1 1 1]
  [1 1 1]]]
Estado atual da Balsa:
[[[0 0 0]
  [0 1 0]
  [0 0 1]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]]]


Movendo container 23 de (4)(0, 1, 1) para (16)(1, 1, 1)
Estabilidade 0.5527707983925667: X->(1.3333333333333333) Y->(1.3333333333333333) Z->(1.0)
Es